<a href="https://colab.research.google.com/github/louamlemjid/project06-humanoid-obj-manipulation/blob/master/humanoid_louamlemjid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Supprimer le dossier repo dans colab !!!

In [ ]:
!rm -r project06-humanoid-obj-manipulation #7ot repo mte3ik win XML files

importer le repo :

In [ ]:
!git clone https://github.com/louamlemjid/project06-humanoid-obj-manipulation.git

load model
**H1**

# Task
Install dm_control and its dependencies, then load the "h1.xml" MuJoCo model, create a physics environment, render the simulation, and display the rendering in the Colab notebook.

## Install dm control

### Subtask:
Install the `dm_control` library and its dependencies.


In [ ]:
#@title Run to install MuJoCo and `dm_control`
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

print('Installing dm_control...')
!pip install -q dm_control>=1.0.31

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')
else:
  del pixels, suite

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")

**Reasoning**:
Install the `dm_control` library and its dependencies using pip.



In [5]:
#@title All `dm_control` imports required for this tutorial

# The basic mujoco wrapper.
from dm_control import mujoco

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib

# Composer high level imports
from dm_control import composer
from dm_control.composer.observation import observable
from dm_control.composer import variation

# Imports for Composer tutorial example
from dm_control.composer.variation import distributions
from dm_control.composer.variation import noises
from dm_control.locomotion.arenas import floors

# Control Suite
from dm_control import suite

# Run through corridor example
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.arenas import corridors as corridor_arenas
from dm_control.locomotion.tasks import corridors as corridor_tasks

# Soccer
from dm_control.locomotion import soccer

# Manipulation
from dm_control import manipulation

## Import necessary libraries

### Subtask:
Import modules from `dm_control` and other required libraries.


In [ ]:
#@title Other imports and helper functions

# General
import copy
import os
import time
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
# Internal loading of video libraries.

# Use svg backend for figure rendering
%config InlineBackend.figure_format = 'svg'

# Font sizes
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Inline video helper function
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
# Access to enums and MuJoCo library functions.
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Seed numpy's global RNG so that cell outputs are deterministic. We also try to
# use RandomState instances that are local to a single cell wherever possible.
np.random.seed(42)

**Reasoning**:
Import the necessary modules from dm_control and matplotlib as instructed.



## Load the mujoco model

### Subtask:
Load your `h1.xml` MuJoCo model using `dm_control`'s loading functions.


**Reasoning**:
Define the model path and load the MuJoCo model using `dm_control`.



In [ ]:
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True
pixels = physics.render(scene_option=scene_option)
PIL.Image.fromarray(pixels)

### **DEBUT DE L'IMPLEMENTATION**

In [7]:
# src/sim/scene_builder.py

from dm_control import mujoco
import os
import PIL.Image
class SceneBuilder:
    """
    Responsible for building a complete MuJoCo scene directly
    from a given XML file.
    """

    @staticmethod
    def build(scene_xml_path: str) -> mujoco.Physics:
        """
        Loads and compiles a MuJoCo scene from a specified XML file.

        This is a static method, so you can call it directly on the class
        without creating an instance: `SceneBuilder.build_from_xml(...)`.

        Args:
            scene_xml_path: The full path to the scene's .xml file.

        Returns:
            A compiled dm_control.mujoco.Physics instance.

        Raises:
            FileNotFoundError: If the provided XML file path does not exist.
        """
        if not os.path.exists(scene_xml_path):
            raise FileNotFoundError(
                f"The specified scene file was not found: {scene_xml_path}"
            )

        print(f"Loading scene from: {scene_xml_path}")

        # The core function to load a model from a complete XML file.
        physics = mujoco.Physics.from_xml_path(scene_xml_path)

        print("Scene built successfully.")
        return physics

# instantiation

In [ ]:
pathToModel = "./project06-humanoid-obj-manipulation/models/dex_hand/cube_scene.xml"
sceneBuilder = SceneBuilder()
physics = sceneBuilder.build(pathToModel)
physics

# visualisation

In [ ]:
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True
pixels = physics.render(scene_option=scene_option)
PIL.Image.fromarray(pixels)

## dex hand class

In [ ]:
physics.named.data.xpos

In [ ]:
physics.named.data.qpos

# Difference entre QPOS et XPOS ?????

In [ ]:
physics.model.actuator_ctrlrange

In [14]:
# src/robot/base_robot.py
from abc import ABC, abstractmethod
import numpy as np
from dm_control import mujoco

class BaseRobot(ABC):
    """Abstract base class for any robot in the simulation."""

    def __init__(self, physics: mujoco.Physics):
        self._physics = physics
        self._model = physics.model
        self._controllable_joint_ids = []
        self._actuator_ids = []

    @property
    def physics(self) -> mujoco.Physics:
        return self._physics

    @property
    def model(self) -> mujoco.wrapper.core.MjModel:
        return self._model

    @property
    def controllable_joint_ids(self) -> list[int]:
        """Returns a list of MuJoCo IDs for joints controlled by this robot."""
        return self._controllable_joint_ids

    @property
    def actuator_ids(self) -> list[int]:
        """Returns a list of MuJoCo IDs for actuators controlling this robot."""
        return self._actuator_ids

    @property
    def action_space_limits(self) -> tuple[np.ndarray, np.ndarray]:
        """Returns (low, high) bounds for the robot's action space."""
        low = self._model.actuator_ctrlrange[self.actuator_ids, 0]
        high = self._model.actuator_ctrlrange[self.actuator_ids, 1]
        return low, high

    @abstractmethod
    def get_observation_data(self) -> np.ndarray:
        """Returns the part of the observation related to this robot's state."""
        pass

    @abstractmethod
    def set_commands(self, actions: np.ndarray):
        """Applies actions to the robot's actuators."""
        pass

    @abstractmethod
    def reset_state(self):
        """Resets the robot's joint positions and velocities to initial state."""
        pass
class DexHandRobot(BaseRobot):
    """Concrete implementation for the Dex Hand robot."""

    def __init__(self, physics: mujoco.Physics):
        super().__init__(physics)
        # Identify specific joint and actuator IDs for the Dex Hand
        # Filter for names starting with 'lh_' (left hand) and exclude wrist for finger-only control
        self._controllable_joint_ids = [5,4,3,2,18,17,16,15,14,9,8,
            7,6,13,12,11,10,23,22,21,20,19]
        self._actuator_ids = [9, 8, 7, 19, 18, 17, 16, 12, 11, 10, 15, 14, 13, 6, 5, 4, 3, 2]


        # Store initial qpos for the Dex Hand for reset (only for controllable joints)
        self._initial_qpos = self.physics.data.qpos[self.controllable_joint_ids].copy()

    def get_observation_data(self) -> np.ndarray:
        """Returns Dex Hand's qpos and qvel for controllable joints."""
        qpos_obs = self.physics.data.qpos[self.controllable_joint_ids]
        qvel_obs = self.physics.data.qvel[self.controllable_joint_ids]
        return np.concatenate([qpos_obs, qvel_obs]).astype(np.float32)

    def set_commands(self, actions: np.ndarray):
        """Applies actions (target joint positions) to Dex Hand actuators."""
        # Actions array should directly correspond to the order of actuator_ids
        for i, act_id in enumerate(self.actuator_ids):
            self.physics.data.ctrl[act_id] = actions[i]

    def reset_state(self):
        """Resets the Dex Hand's joint positions and velocities."""
        # Set qpos for controllable joints to their initial state
        self.physics.data.qpos[self.controllable_joint_ids] = self._initial_qpos
        # Set qvel for controllable joints to zero
        self.physics.data.qvel[self.controllable_joint_ids] = 0.0

# instantiation dex hand

In [ ]:
robot = DexHandRobot(physics)
robot.get_observation_data()

## object class

In [16]:
# src/objects/base_object.py
from abc import ABC, abstractmethod
import numpy as np
from dm_control import mujoco

class BaseObject(ABC):
    """Abstract base class for any manipulable object."""

    def __init__(self, physics: mujoco.Physics, object_config: dict):
        self._physics = physics
        self._model = physics.model
        self._object_config = object_config
        self._body_id = self._model.name2id('cube','body')
        self._joint_id = self._model.name2id('cube_joint','joint')

    @property
    def physics(self) -> mujoco.Physics:
        return self._physics

    @property
    def model(self) -> mujoco.wrapper.core.MjModel:
        return self._model

    @property
    def body_id(self) -> int:
        return self._body_id

    @property
    def joint_id(self) -> int:
        return self._joint_id

    @abstractmethod
    def get_observation_data(self) -> np.ndarray:
        """Returns the part of the observation related to this object's state."""
        pass

    @abstractmethod
    def reset_state(self):
        """Resets the object's position and orientation to an initial (possibly randomized) state."""
        pass

# src/objects/cube_object.py
import numpy as np
from dm_control import mujoco
class CubeObject(BaseObject):
    """Concrete implementation for a manipulable cube object."""

    def __init__(self, physics: mujoco.Physics, object_config: dict):
        super().__init__(physics, object_config)
        self._initial_pos_relative = np.array(object_config["initial_pos_relative"])
        self._pos_randomization_range = {
            k: np.array(v) for k, v in object_config["pos_randomization_range"].items()
        }

    def get_observation_data(self) -> np.ndarray:
        """Returns cube's position, orientation, linear and angular velocities."""
        object_pos = self._physics.data.xpos[self._body_id]
        object_quat = self._physics.data.xquat[self._body_id]
        object_lin_vel = self._physics.data.object_velocity(self.body_id,'body')[0]
        object_ang_vel = self._physics.data.object_velocity(self.body_id,'body')[1]
        return np.concatenate([object_pos, object_quat, object_lin_vel, object_ang_vel]).astype(np.float32)

    def reset_state(self):
        """Resets the cube's position to initial (with randomization) and velocity to zero."""
        object_initial_qpos_idx = self._model.jnt_qposadr[self.joint_id]

        # Apply randomization to initial position
        random_offset_x = np.random.uniform(self._pos_randomization_range['x'][0], self._pos_randomization_range['x'][1])
        random_offset_y = np.random.uniform(self._pos_randomization_range['y'][0], self._pos_randomization_range['y'][1])
        random_offset_z = np.random.uniform(self._pos_randomization_range['z'][0], self._pos_randomization_range['z'][1])

        new_pos = self._initial_pos_relative + np.array([random_offset_x, random_offset_y, random_offset_z])

        # Set the object's position (first 3 values of its free joint qpos)
        self._physics.data.qpos[object_initial_qpos_idx:object_initial_qpos_idx+3] = new_pos
        # Set the object's orientation (quaternion, next 4 values). Keep it upright.
        self._physics.data.qpos[object_initial_qpos_idx+3:object_initial_qpos_idx+7] = np.array([1.0, 0.0, 0.0, 0.0]) # WXYZ identity quaternion

        # Reset object's velocities (6 values for a free joint)
        object_initial_qvel_idx = self._model.jnt_dofadr[self.joint_id]
        self._physics.data.qvel[object_initial_qvel_idx:object_initial_qvel_idx+6] = 0.0

## Instantiation Object

In [17]:
object_config = {
    "initial_pos_relative": [0.0, 0.0, 0.0],
    "pos_randomization_range": {
        "x": [-0.1, 0.1],
        "y": [-0.02, 0.02],
    "z": [-0.005, 0.005]},
    "name": "cube" ,
  "joint_name": "cube_joint" ,
  "geom_name": "cube_geom",
  "size": [0.03, 0.04, 0.02],
    "rewards":{
            "object_height_factor": 100,
                "target_height": 0.06,
                "target_height_bonus": 500,
                "drop_penalty": 200
            },
       "env":{
           "hand_model_path": "models/dex_hand/scene.xml",
           "control_timestep": 0.002,
           "episode_duration": 2.0 # seconds
       },
    "rl":
    {
      "total_timesteps": 50000 ,# Keep low for initial test, increase to millions for actual training
      "log_dir": "./sac_hand_manipulation_logs/",
      "model_save_path": "./trained_models/hand_policy.zip",
      "learning_rate": 0.0003,
      "buffer_size": 10000,
      "learning_starts": 100, # Start learning quickly for test
      "train_freq": [1, "episode"],
      "gradient_steps": 1,
      "ent_coef": "auto",
      "save_freq": 5000, # How often to save checkpoints during training}
    }}

manipulable_object = CubeObject(physics, object_config)

In [ ]:
manipulable_object.get_observation_data()

### observation builder

In [19]:
# src/env/observation_builder.py
import numpy as np


class ObservationBuilder:
    """
    Responsible for building the complete observation array from various components.
    """
    def __init__(self, robot: BaseRobot, manipulable_object: BaseObject):
        self._robot = robot
        self._object = manipulable_object

    def build_observation(self) -> np.ndarray:
        """Collects observation data from robot and object and concatenates them."""
        robot_obs = self._robot.get_observation_data()
        object_obs = self._object.get_observation_data()

        obs = np.concatenate([
            robot_obs,
            object_obs,
        ]).astype(np.float32)
        return obs

## Instantiaiton observation builder

In [ ]:
 observation_builder = ObservationBuilder(robot, manipulable_object)
 observation_builder.build_observation()

## reward component

In [21]:
physics.data.xpos[manipulable_object.body_id]

array([ 0.38657388,  1.44734049, -0.07510776])

In [22]:
# src/env/reward_components.py
import numpy as np
from dm_control import mujoco

# Define individual reward functions here.
# Each function takes physics, relevant object_body_id, and config.

def object_height_reward(physics: mujoco.Physics, object_body_id: int, config: dict) -> float:
    """Rewards the agent based on the object's height."""
    object_height = physics.data.xpos[object_body_id][2] # Z-coordinate
    reward = config.get("object_height_factor", 100) * (object_height - 0.05) # Assume table is at Z=0.05
    return float(reward)

def target_height_bonus_reward(physics: mujoco.Physics, object_body_id: int, config: dict) -> float:
    """Gives a bonus if the object reaches a target height."""
    object_height = physics.data.xpos[object_body_id][2]
    if object_height >= config.get("target_height", 0.06):
        return float(config.get("target_height_bonus", 500))
    return 0.0

def drop_penalty_reward(physics: mujoco.Physics, object_body_id: int, config: dict) -> float:
    """Penalizes the agent if the object falls too low."""
    object_height = physics.data.xpos[object_body_id][2]
    if object_height < config.get("drop_threshold", 0.02): # Below a very low threshold
        return float(-config.get("drop_penalty", 200))
    return 0.0

## Reward Composer

In [23]:
# src/env/reward_composer.py
from typing import Callable, List
from dm_control import mujoco

class RewardComposer:
    """
    Composes multiple individual reward components into a single total reward.
    """
    def __init__(self, physics: mujoco.Physics, object_body_id: int, reward_config: dict):
        self._physics = physics
        self._object_body_id = object_body_id
        self._reward_config = reward_config

        # Define the list of active reward functions to use
        self._reward_functions: List[Callable[[mujoco.Physics, int, dict], float]] = [
            object_height_reward,
            target_height_bonus_reward,
            drop_penalty_reward,
        ]

    def get_total_reward(self) -> float:
        """Calculates the sum of all active reward components."""
        total_reward = 0.0
        for func in self._reward_functions:
            total_reward += func(self._physics, self._object_body_id, self._reward_config)
        return total_reward

# Instantiation Reward Composer

In [ ]:
reward_composer = RewardComposer(physics, manipulable_object.body_id, object_config["rewards"])
reward_composer.get_total_reward()

## Hand manipulation environment

In [ ]:
robot.action_space_limits

In [26]:
# src/env/hand_manipulation_env.py
import gymnasium as gym
from gymnasium import spaces
from dm_control import mujoco
import numpy as np

class HandManipulationEnv(gym.Env):
    """
    Gymnasium environment for Dex Hand object manipulation.
    Orchestrates the robot, object, observation, and reward components.
    """
    def __init__(self, physics: mujoco.Physics,
                 robot: BaseRobot,
                 manipulable_object: BaseObject,
                 observation_builder: ObservationBuilder,
                 reward_composer: RewardComposer,
                 env_config: dict):

        super().__init__()
        self._physics = physics
        self._model = physics.model
        self._robot = robot
        self._object = manipulable_object
        self._observation_builder = observation_builder
        self._reward_composer = reward_composer
        self._env_config = env_config

        self.control_timestep = self._env_config["control_timestep"]
        self.episode_duration = self._env_config["episode_duration"]

        # Define Observation Space based on the ObservationBuilder
        dummy_obs = self._observation_builder.build_observation()
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=dummy_obs.shape, dtype=np.float32)

        # Define Action Space based on the robot's action space limits
        low_action, high_action = self._robot.action_space_limits
        self.action_space = spaces.Box(low=low_action, high=high_action, dtype=np.float32)

        # Store full initial qpos/qvel for MuJoCo's direct reset,
        # then let robot/object components handle their specific parts.
        self._initial_qpos_all = self._physics.data.qpos.copy()
        self._initial_qvel_all = self._physics.data.qvel.copy()


    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self._physics.reset() # Reset MuJoCo's internal state to model defaults

        # Reset all qpos/qvel to known initial states (important for deterministic restarts)
        self._physics.data.qpos[:] = self._initial_qpos_all
        self._physics.data.qvel[:] = self._initial_qvel_all

        # Delegate reset logic to specific components
        self._robot.reset_state()
        self._object.reset_state()

        # Step physics a few times to stabilize after reset
        for _ in range(10): # Simulate 10 steps to allow physics to settle
            self._physics.step()

        observation = self._observation_builder.build_observation()
        info = {}
        return observation, info

    def step(self, action):
        # Clip actions to stay within the action space boundaries
        clipped_action = np.clip(action, self.action_space.low, self.action_space.high)

        # Apply actions to the robot's actuators
        self._robot.set_commands(clipped_action)

        # Simulate for 'num_substeps' to cover 'control_timestep' duration
        num_substeps = int(self.control_timestep / self._model.opt.timestep)
        for _ in range(num_substeps):
            self._physics.step()

        reward = self._reward_composer.get_total_reward()

        # Check for episode termination (time limit)
        terminated = False
        if self._physics.data.time >= self.episode_duration:
            terminated = True

        truncated = False # In gymnasium, time limits typically result in 'truncated=True' but for simplicity initially, 'terminated=True' works.

        observation = self._observation_builder.build_observation()
        info = {}

        return observation, reward, terminated, truncated, info

    def render(self):
        # Render the MuJoCo scene for visualization
        return self._physics.render(width=640, height=480, camera_id=0) # You might need to adjust camera_id

    def close(self):
        # No explicit close needed for dm_control.Physics, but good practice
        pass

# Instantiation ENV GYM

In [ ]:
env = HandManipulationEnv(
        physics=physics,
        robot=robot,
        manipulable_object=manipulable_object,
        observation_builder=observation_builder,
        reward_composer=reward_composer,
        env_config=object_config["env"]
    )

## **Stabe Baseline3 class**

In [ ]:
!pip install -q stable_baselines3[extra]

In [29]:
# src/rl/rl_agent_manager.py
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env # Not used in this minimal test, but useful
from stable_baselines3.common.callbacks import CheckpointCallback
import os
import gymnasium as gym

class RLAgentManager:
    """Manages the lifecycle of an RL agent (training, saving, loading)."""

    def __init__(self, env: gym.Env, rl_config: dict):
        self._env = env
        self._rl_config = rl_config
        self._model = None

        os.makedirs(self._rl_config["log_dir"], exist_ok=True)
        os.makedirs(os.path.dirname(self._rl_config["model_save_path"]), exist_ok=True)

    def train_agent(self):
        """Initializes and trains the SAC agent."""
        print("Initializing SAC agent...")

        self._model = SAC(
            "MlpPolicy",
            self._env,
            verbose=1,
            learning_rate=self._rl_config["learning_rate"],
            buffer_size=self._rl_config["buffer_size"],
            learning_starts=self._rl_config["learning_starts"],
            train_freq=tuple(self._rl_config["train_freq"]),
            gradient_steps=self._rl_config["gradient_steps"],
            ent_coef=self._rl_config["ent_coef"],
            tensorboard_log=self._rl_config["log_dir"]
        )

        # Callbacks
        checkpoint_callback = CheckpointCallback(
            save_freq=self._rl_config["save_freq"],
            save_path=os.path.join(self._rl_config["log_dir"], "checkpoints"),
            name_prefix='sac_model'
        )

        print(f"Starting training for {self._rl_config['total_timesteps']} timesteps...")
        self._model.learn(
            total_timesteps=self._rl_config["total_timesteps"],
            progress_bar=True,
            callback=[checkpoint_callback]
        )
        print("Training finished!")
        self._model.save(self._rl_config["model_save_path"])
        print(f"Model saved to {self._rl_config['model_save_path']}")

    def evaluate_agent(self, num_episodes: int = 1, render: bool = True):
        """Evaluates the loaded agent."""
        if self._model is None:
            print("No model loaded. Please train or load an agent first.")
            return

        print(f"Evaluating agent for {num_episodes} episodes...")
        for i in range(num_episodes):
            obs, info = self._env.reset()
            done = False
            total_reward = 0
            frames = []
            while not done:
                action, _states = self._model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = self._env.step(action)
                total_reward += reward
                if render:
                    frames.append(self._env.render())
                done = terminated or truncated
            print(f"Episode {i+1}: Total Reward = {total_reward:.2f}")
            # For Colab, you'd typically save frames and then display them as a video.
            # Example (if you have a helper function like display_video):
            # if render and frames:
            #     from your_utils_file import display_video
            #     display_video(frames, fps=1/self._env.control_timestep)
        print("Evaluation complete.")

## **instantiation**

In [ ]:
rl_manager = RLAgentManager(env, object_config["rl"])
print("RL Agent Manager initialized.")

# --- Action: Train the agent ---
rl_manager.train_agent()

Load training

In [ ]:
model_path = "./trained_models/hand_policy.zip"
model = SAC.load(model_path, env=env)
print(f"Model loaded from {model_path}")

Video after training

In [ ]:
# Imports
import numpy as np
from stable_baselines3 import SAC
from IPython.display import HTML
from base64 import b64encode
import matplotlib.pyplot as plt

# Helper function to display video in Colab
def display_video(frames, framerate=30):
    import imageio
    from pathlib import Path
    import io

    video_path = "/tmp/simulation.mp4"
    imageio.mimsave(video_path, frames, fps=framerate)

    mp4 = open(video_path, 'rb').read()
    b64 = b64encode(mp4).decode('ascii')
    html = f'''
      <video width=640 height=480 controls>
            <source src="data:video/mp4;base64,{b64}" type="video/mp4">
      </video>
    '''
    return HTML(html)

# Setup
duration = 4      # seconds
framerate = 30    # Hz
max_steps = duration * framerate

# Load the trained model
model_path = "./trained_models/hand_policy.zip"
model = SAC.load(model_path, env=env)
print(f"✅ Model loaded from: {model_path}")

# Reset environment
obs, info = env.reset()
frames = []

# For rendering with joint visualization
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True

# Collect frames
for _ in range(max_steps):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    # Render and store frame
    pixels = physics.render(scene_option=scene_option)
    frames.append(pixels)

    if terminated or truncated:
        print("Episode finished early.")
        break

# Display video
display_video(frames, framerate)